### Initial Setup

 - initialize the repositories and MongoDB indexes [see commands here](../ToolBox/Repository-Cleanup.ipynb)
    - for `us-east` and `us-west`

### Import in US-EAST

In [9]:
# import in synchronous mode
!import.sh -o import -l import/states-hierarchy-us-east -r us-east -b /

Using config /nxbench/notebooks/11B-Steps/nuxeo.properties
url=http://127.0.0.1:8080/nuxeo
login=nco-admin
Nuxeo Client configured
Connected to Nuxeo Server 11.3.26
Running Operation:StreamImporter.runDocumentConsumersEx
   nbThreads: 10 
   logName: import/states-hierarchy-us-east 
   blockDefaultSyncListeners: true 
   rootFolder: / 
   logSize: 8 
   batchSize: 500 
#####################
Execution completed
elapsed:60.039
committed:26
failures:0
consumers:8
throughput:0.4330518496310731



In [10]:
# import customers
nbDocs = 89997827
expectedThroughput = 14000
print("expected duration (s) =", round(nbDocs/expectedThroughput))
print("expected duration (h) =", round((nbDocs/expectedThroughput)/3600))

expected duration (s) = 6428
expected duration (h) = 2


In [11]:
!import.sh -o import -t 16 -l import/customers-us-east -r us-east -b / -a -w 8000 -bulk > import-useast-customers.log

In [12]:
!tail import-useast-customers.log

Nuxeo Client configured
...........................................................................................................................
Running completed
elapsed:7427.235
committed:90918788
failures:0
consumers:16
throughput:12241.269866915482

Exit after 7443 s


In [13]:
# import accounts
nbDocs = 89997827*2
expectedThroughput = 10000
print("expected duration (s) =", round(nbDocs/expectedThroughput))
print("expected duration (h) =", round((nbDocs/expectedThroughput)/3600))

expected duration (s) = 18000
expected duration (h) = 5


In [ ]:
!import.sh -o import -t 16 -l import/accounts-us-east -r us-east -b / -a -w 25000 -bulk > import-useast-accounts.log

The command exited in timeout since the throughput was not at all what was expected.

    Consumers status: threads: 16, failure 0, 
    messages committed: 181842842, elapsed: 59763.90s, 
    throughput: 3042.69 msg/s


In [ ]:
# import statements
nbDocs = 89997827*6
expectedThroughput = 9000
print("expected duration (s) =", round(nbDocs/expectedThroughput))
print("expected duration (h) =", round((nbDocs/expectedThroughput)/3600))

In [ ]:
!import.sh -o import -t 16 -l import/statements_live-us-east -r us-east -b / -a -w 200000 -bulk > import-useast-live-statements.log

Looks like the importer was stuck at some point because it was started with 16 threads whereas there are 24 partitions.

Looking at the offsets:

    > stream.sh lag -k --verbose -l import/statements_live-us-east
    ## Log: Name{id='import-statements_live-us-east', urn='import/statements_live-us-east'} partitions: 24
    ### Group: Name{id='StreamImporter-runDocumentConsumersEx', urn='StreamImporter/runDocumentConsumersEx'}
| partition | lag | pos | end | posOffset |?endOffset?|
| --- | ---: | ---: | ---: | ---: | ---: |
|All|134614904|410913682|545528586|13325484|22937322|
|0|9497160|13440162|22937322|13440162|22937322|
|1|9478898|13401838|22880736|13401838|22880736|
|2|9460836|13447068|22907904|13447068|22907904|
|3|9480060|13419108|22899168|13419108|22899168|
|4|9494178|13436388|22930566|13436388|22930566|
|5|9545021|13383589|22928610|13383589|22928610|
|6|9418620|13426782|22845402|13426782|22845402|
|7|9522299|13385569|22907868|13385569|22907868|
|8|9239384|13390192|22629576|13390192|22629576|
|9|9324698|13333324|22658022|13333324|22658022|
|10|9275058|13387290|22662348|13387290|22662348|
|11|9314988|13325484|22640472|13325484|22640472|
|12|5352228|17275308|22627536|17275308|22627536|
|13|5406157|17226965|22633122|17226965|22633122|
|14|5364081|17273745|22637826|17273745|22637826|
|15|5441238|17230662|22671900|17230662|22671900|
|16|0|22613778|22613778|22613778|22613778|
|17|0|22633500|22633500|22633500|22633500|
|18|0|22629768|22629768|22629768|22629768|
|19|0|22640196|22640196|22640196|22640196|
|20|0|22680138|22680138|22680138|22680138|
|21|0|22658220|22658220|22658220|22658220|
|22|0|22667322|22667322|22667322|22667322|
|23|0|22607286|22607286|22607286|22607286|

So clearly:

 - 8 threads went to the end
 - then the problems started

Tried to stop/restart, but the same phenomena did happen:

<img src="monitoring/us-east-import-glitch.png"/>

In [19]:
!import.sh -o import -t 24 -l import/statements_live-us-east -r us-east -b / -a -w 200000 -bulk > import-useast-live-statements-part2.log

Exception in thread "main" org.nuxeo.client.spi.NuxeoClientException: Error during call on url=http://127.0.0.1:8080/nuxeo/site/api/v1/automation/StreamImporter.runDocumentConsumersEx/@async/a191921f-297f-451f-8298-be4e0d352922/status
	at org.nuxeo.client.NuxeoClient.request(NuxeoClient.java:261)
	at org.nuxeo.client.NuxeoClient.get(NuxeoClient.java:233)
	at org.nuxeo.data.gen.cli.NuxeoClientHelper.waitForResult(NuxeoClientHelper.java:105)
	at org.nuxeo.data.gen.cli.RestCli.main(RestCli.java:226)
Caused by: java.net.ConnectException: Failed to connect to /127.0.0.1:8080
	at okhttp3.internal.connection.RealConnection.connectSocket(RealConnection.java:248)
	at okhttp3.internal.connection.RealConnection.connect(RealConnection.java:166)
	at okhttp3.internal.connection.StreamAllocation.findConnection(StreamAllocation.java:257)
	at okhttp3.internal.connection.StreamAllocation.findHealthyConnection(StreamAllocation.java:135)
	at okhttp3.internal.connection.StreamAllocation.newStream(StreamAll

### Indexing US-EAST

 - scale out the number of Nuxeo Worker nodes
 - configure ES for bulk indexing (see [ES toolbox](../ToolBox/Elasticsearch.ipynb))
     - no replicas
     - refresh rate
     
 Start Indexing on US-EAST using BAF    

In [27]:
!(INJECTOR="http://127.0.0.1:8080";\
curl -H 'Content-Type:application/json+nxrequest' \
  -H 'X-NXRepository:us-east' \
  -X POST -d '{"params":{},"context":{}}' -u $NXUSER:$NXPWD \
  "$INJECTOR/nuxeo/api/v1/automation/Elasticsearch.BulkIndex")

curl: (7) Failed to connect to 127.0.0.1 port 8080: Connection refused


In [29]:
# check status
! cid="6057b8cb-c073-466c-96c5-9252aee168cc"; \
  INJECTOR="http://127.0.0.1:8080"; \
  curl -H 'Content-Type:application/json+nxrequest' -H 'X-NXRepository:us-east' \
  -u $NXUSER:$NXPWD $INJECTOR/nuxeo/api/v1/bulk/$cid

curl: (7) Failed to connect to 127.0.0.1 port 8080: Connection refused


### Import in US-WEST

In [20]:
# import in synchronous mode
!import.sh -o import -l import/states-hierarchy-us-west -r us-west -b /

Using config /nxbench/notebooks/11B-Steps/nuxeo.properties
url=http://127.0.0.1:8080/nuxeo
login=nco-admin
Nuxeo Client configured
Connected to Nuxeo Server 11.3.26
Running Operation:StreamImporter.runDocumentConsumersEx
   nbThreads: 10 
   logName: import/states-hierarchy-us-west 
   blockDefaultSyncListeners: true 
   rootFolder: / 
   logSize: 8 
   batchSize: 500 
#####################
Execution completed
elapsed:60.041
committed:26
failures:0
consumers:8
throughput:0.43303742442664184



In [ ]:
!import.sh -o import -t 16 -l import/customers-us-west -r us-west -b / -a -w 8000 -bulk > import-uswest-customers.log

In [ ]:
!import.sh -o import -t 16 -l import/accounts-us-west -r us-west -b / -a -w 40000 -bulk > import-uswest-accounts.log

In [ ]:
!import.sh -o import -t 24 -l import/statements_live-us-west -r us-west -b / -a -w 80000 -bulk > import-uswest-live-statements.log

### Indexing US-WEST
 
 - configure ES for bulk indexing (see [ES toolbox](../ToolBox/Elasticsearch.ipynb))
     - no replicas
     - refresh rate
     
Start Indexing on US-WEST using BAF    

In [ ]:
!(INJECTOR="http://127.0.0.1:8080";\
curl -H 'Content-Type:application/json+nxrequest' \
  -H 'X-NXRepository:us-west' \
  -X POST -d '{"params":{},"context":{}}' -u $NXUSER:$NXPWD \
  "$INJECTOR/nuxeo/api/v1/automation/Elasticsearch.BulkIndex")

In [ ]:
# check status
! cid="6057b8cb-c073-466c-96c5-9252aee168cc"; \
  INJECTOR="http://127.0.0.1:8080"; \
  curl -H 'Content-Type:application/json+nxrequest' -H 'X-NXRepository:us-west' \
  -u $NXUSER:$NXPWD $INJECTOR/nuxeo/api/v1/bulk/$cid